# Phi-4 Mini ONNX Parallel Function Calling Tutorial

บทเรียนนี้แสดงวิธีการใช้ Phi-4 Mini ร่วมกับ ONNX Runtime GenAI สำหรับการเรียกใช้งานฟังก์ชันแบบขนาน การเรียกใช้งานฟังก์ชันช่วยให้โมเดลสามารถเรียกใช้เครื่องมือและ API ภายนอกได้อย่างชาญฉลาดตามคำขอของผู้ใช้

## ภาพรวม

ในบทเรียนนี้ คุณจะได้เรียนรู้วิธี:
- ตั้งค่า Phi-4 Mini ร่วมกับ ONNX Runtime GenAI
- กำหนดโครงสร้างฟังก์ชันสำหรับการจองเที่ยวบินและโรงแรม
- ใช้การสร้างผลลัพธ์แบบมีไกด์ด้วยไวยากรณ์ Lark เพื่อให้ได้ผลลัพธ์ที่มีโครงสร้าง
- ดำเนินการเรียกใช้งานฟังก์ชันแบบขนานสำหรับสถานการณ์การจองการเดินทางที่ซับซ้อน

## สิ่งที่ต้องเตรียม

ก่อนที่จะเริ่มต้นบทเรียนนี้ ตรวจสอบให้แน่ใจว่าคุณได้:
- ดาวน์โหลดโมเดล Phi-4 Mini ONNX
- ติดตั้งแพ็กเกจ `onnxruntime-genai`
- มีความเข้าใจพื้นฐานเกี่ยวกับแนวคิดการเรียกใช้งานฟังก์ชัน


## ขั้นตอนที่ 1: นำเข้าไลบรารีที่จำเป็น

เริ่มต้นด้วยการนำเข้าไลบรารีที่จำเป็นสำหรับการใช้งานฟังก์ชันของเรา


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## ขั้นตอนที่ 2: การตั้งค่าและการกำหนดค่าของโมเดล

ตอนนี้เราจะตั้งค่าโมเดล Phi-4 Mini ONNX ตรวจสอบให้แน่ใจว่าได้เปลี่ยนเส้นทางเป็นไดเรกทอรีโมเดลจริงของคุณ


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## ขั้นตอนที่ 3: กำหนดค่าพารามิเตอร์การสร้าง

ตั้งค่าพารามิเตอร์การสร้างเพื่อควบคุมพฤติกรรมการตอบสนองของโมเดล การตั้งค่าเหล่านี้ช่วยให้การเรียกใช้งานฟังก์ชันมีความแน่นอนและมุ่งเน้นมากขึ้น


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## ขั้นตอนที่ 4: กำหนดฟังก์ชันที่สามารถใช้งานได้

ในขั้นตอนนี้ เราจะกำหนดฟังก์ชันที่ผู้ช่วย AI ของเราสามารถเรียกใช้งานได้ ในตัวอย่างนี้ เรามีฟังก์ชันที่เกี่ยวข้องกับการเดินทางสองฟังก์ชัน:
1. **booking_flight_tickets**: สำหรับการจองตั๋วเครื่องบินระหว่างสนามบิน
2. **booking_hotels**: สำหรับการจองที่พักในโรงแรม

การกำหนดฟังก์ชันจะเป็นไปตามรูปแบบ schema การเรียกฟังก์ชันของ OpenAI


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## ขั้นตอนที่ 5: ฟังก์ชันช่วยสร้างไวยากรณ์

ฟังก์ชันช่วยเหล่านี้จะแปลงคำจำกัดความของฟังก์ชันของเราให้อยู่ในรูปแบบไวยากรณ์ของ Lark ซึ่งถูกใช้โดย ONNX Runtime GenAI เพื่อการสร้างที่มีการกำกับดูแล สิ่งนี้ช่วยให้โมเดลสร้างการเรียกใช้ฟังก์ชันที่ถูกต้องในรูปแบบ JSON ที่เหมาะสม


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## ขั้นตอนที่ 6: ทดสอบการสร้างไวยากรณ์

มาทดสอบฟังก์ชันการสร้างไวยากรณ์ของเราเพื่อดูว่ามันแปลงคำจำกัดความของเครื่องมือไปเป็นรูปแบบที่เหมาะสมได้อย่างไร


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## ขั้นตอนที่ 7: เตรียมข้อความระบบและตัวสร้าง

ตอนนี้เราจะสร้างข้อความระบบที่บอกโมเดลเกี่ยวกับเครื่องมือที่มีอยู่ และตั้งค่าตัวสร้างด้วยพารามิเตอร์การสร้างที่มีการแนะนำ


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## ขั้นตอนที่ 8: เริ่มต้นสร้าง Generator ด้วยการสร้างแบบมีคำแนะนำ

ตอนนี้เราจะสร้างตัว generator ด้วยพารามิเตอร์ที่ตั้งค่าไว้ และใช้ไวยากรณ์ Lark เพื่อการสร้างแบบมีคำแนะนำ


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## ขั้นตอนที่ 9: ทดสอบการเรียกฟังก์ชันแบบขนาน

ตอนนี้เรามาทดสอบการตั้งค่าของเราด้วยสถานการณ์การจองการเดินทางที่ซับซ้อนซึ่งต้องเรียกใช้ฟังก์ชันหลายตัว


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### สร้างการเรียกใช้งานฟังก์ชัน

โมเดลจะสร้างการเรียกใช้งานฟังก์ชันที่มีโครงสร้างตามคำขอของผู้ใช้ โดยการสร้างที่มีการกำกับจะทำให้ผลลัพธ์อยู่ในรูปแบบ JSON ที่ถูกต้องและสามารถนำไปใช้งานได้โดยตรง

**ผลลัพธ์ที่คาดหวัง**: โมเดลควรสร้างการเรียกใช้งานฟังก์ชันสำหรับ:
1. `booking_flight_tickets` พร้อมรายละเอียดเที่ยวบินจากปักกิ่ง (PEK) ไปปารีส (CDG)
2. `booking_hotels` พร้อมรายละเอียดที่พักในปารีส

รันเซลด้านล่างเพื่อดูการสร้างแบบสด:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## สรุป

🎉 **ยินดีด้วย!** คุณได้ดำเนินการเรียกฟังก์ชันแบบขนานด้วย Phi-4 Mini โดยใช้ ONNX Runtime GenAI สำเร็จแล้ว

### สิ่งที่คุณได้เรียนรู้:

1. **การตั้งค่ารุ่น**: วิธีการตั้งค่า Phi-4 Mini ด้วย ONNX Runtime GenAI  
2. **การกำหนดฟังก์ชัน**: วิธีการกำหนดโครงสร้างฟังก์ชันสำหรับการเรียกฟังก์ชัน AI  
3. **การสร้างผลลัพธ์แบบมีโครงสร้าง**: วิธีใช้ไวยากรณ์ Lark เพื่อสร้างผลลัพธ์ที่มีโครงสร้าง  
4. **การเรียกฟังก์ชันแบบขนาน**: วิธีจัดการคำขอที่ซับซ้อนซึ่งต้องการการเรียกฟังก์ชันหลายครั้ง  

### ประโยชน์สำคัญ:

- ✅ **ผลลัพธ์ที่มีโครงสร้าง**: การสร้างผลลัพธ์แบบมีโครงสร้างช่วยให้มั่นใจได้ว่าการเรียกฟังก์ชัน JSON มีความถูกต้อง  
- ✅ **การประมวลผลแบบขนาน**: จัดการการเรียกฟังก์ชันหลายครั้งในคำขอเดียว  
- ✅ **ประสิทธิภาพสูง**: ONNX Runtime ให้การอนุมานที่ปรับแต่งมาอย่างดี  
- ✅ **โครงสร้างที่ยืดหยุ่น**: เพิ่มหรือแก้ไขการกำหนดฟังก์ชันได้ง่าย  

### แหล่งข้อมูล:

- [Phi-4 Mini Documentation](https://huggingface.co/microsoft/Phi-4-mini-onnx)  
- [ONNX Runtime GenAI Documentation](https://onnxruntime.ai/docs/genai/)  
- [Function Calling Best Practices](https://platform.openai.com/docs/guides/function-calling)  



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามืออาชีพ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดที่เกิดจากการใช้การแปลนี้
